In [1]:
import pickle

import pandas as pd
from statsbombpy import sb
from mplsoccer import Pitch
import pandas as pd
from one_two.functions import get_pass_data, get_one_twos

In [2]:
# load in all WSL 20/21 passes
com_id = 37
season_id = 90
path = ""

comp_df = sb.matches(competition_id=com_id, season_id=season_id)

In [ ]:
# get all pass data

all_passes = pd.DataFrame([])
teams = comp_df.home_team.unique()

for side in teams:
    team_passes = get_pass_data(competition_ID=com_id, season_ID=season_id, team=side, data_path=path)

    all_passes = pd.concat([all_passes, team_passes], axis=0)


In [ ]:
all_passes.shape

In [ ]:
# then get only one-twos from this season


In [ ]:
# then compute statistics of shots/assists from these one-twos